In [1]:
from cwms_read import get_cwms
import pandas as pd
from bokeh.layouts import row, widgetbox, gridplot
from bokeh.models import Select
from bokeh.io import show, output_notebook
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from smooth import decompose 
from plots import bok_decompose, bok_line
from boxplot import bokeh_ts_bx_plt

In [2]:
output_notebook()
paths = ['TDDO.Temp-Water.Inst.1Hour.0.GOES-REV', 'LGNW.Temp-Water.Inst.1Hour.0.GOES-REV']
feature_names = ['TDDO_Temp-Water', 'LGNW_Temp-Water']
data = get_cwms(paths, interval = 'hourly', lookback = 10)
decomp_dict = {'TDDO_Temp-Water': decompose(data['TDDO_Temp-Water'].interpolate().dropna(), frequency= 24, s_window = 'periodic', log = True),
               'LGNW_Temp-Water':decompose(data['LGNW_Temp-Water'].interpolate().dropna(), frequency= 24, s_window = 'periodic', log = True)}

# Add in dictionary for the boxplot

Loading BokehJS ...

In [7]:
def modify_doc(doc):
    def create_figure():
        current_feature_name = feature_name.value
        decomp = decomp_dict[current_feature_name]
        p1 = bok_decompose(decomp)
        series = data[current_feature_name]
        p2 = bokeh_ts_bx_plt(series, current_feature_name,freq = 'week')
        p = row(p1,p2)
        return p
    
    def update(attr, old, new):
        layout.children[1] = create_figure()
        
    
    # Controls
    feature_name = Select(title="Site", options=feature_names, value=feature_names[0])
    feature_name.on_change('value', update)
    controls = widgetbox([feature_name], width=200)
    p = create_figure()
    layout = row(controls, p)
    doc.add_root(layout)
    



In [8]:
# Set up the Application 
handler = FunctionHandler(modify_doc)
app = Application(handler)
# Create the Document
# Not strictly necessary, but helps w/ debugging
doc = app.create_document()
# Show the application
# Make sure the URL matches your Jupyter instance
show(app, notebook_url="localhost:8888")
